In [11]:
import pandas as pd 
import numpy as np
from qiime2 import Artifact
import os

Import of the final sample dataset obtained in the EDA notebook: 

In [12]:
data = pd.read_csv('data_final_groups.csv',dtype = {"sample_name": "str"},index_col=0)
data

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency,group
0,10317.000023139,No,No,No,No,60s,69.0,69.0,Yes,Rarely (a few times/month),...,Regularly (3-5 times/week),Daily,Never,NaN,Remained stable,63,kilograms,Regularly (3-5 times/week),Regularly (3-5 times/week),Treatment
1,10317.000027811,No,No,No,No,40s,46.0,46.0,Yes,Rarely (a few times/month),...,Rarely (a few times/month),Regularly (3-5 times/week),Rarely (a few times/month),NaN,Remained stable,81,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
2,10317.000031513,Self-diagnosed,No,No,Not provided,30s,37.0,37.0,Yes,Regularly (3-5 times/week),...,Never,Rarely (a few times/month),Rarely (a few times/month),NaN,Remained stable,49,kilograms,Occasionally (1-2 times/week),Rarely (less than once/week),Treatment
3,10317.000033280,No,No,No,No,60s,60.0,60.0,Yes,Regularly (3-5 times/week),...,Never,Never,Never,NaN,Remained stable,68,kilograms,Occasionally (1-2 times/week),Regularly (3-5 times/week),Treatment
4,10317.000038261,No,No,No,No,40s,48.0,48.0,Yes,Regularly (3-5 times/week),...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,70,kilograms,Rarely (less than once/week),Occasionally (1-2 times/week),Treatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,10317.000065667,No,No,No,No,20s,22.0,22.0,Yes,Occasionally (1-2 times/week),...,Never,Never,Rarely (a few times/month),NaN,Remained stable,61,kilograms,Regularly (3-5 times/week),Never,Control
376,10317.000065673,No,No,Yes,No,20s,20.0,20.0,No,Never,...,Never,Never,Occasionally (1-2 times/week),NaN,Remained stable,79,kilograms,Never,Rarely (less than once/week),Control
377,10317.000065675,No,No,No,No,20s,21.0,21.0,No,Never,...,Never,Never,Daily,NaN,Remained stable,51,kilograms,Never,Regularly (3-5 times/week),Control
378,10317.000065681,No,No,Yes,No,20s,21.0,21.0,No,Never,...,Never,Never,Rarely (a few times/month),NaN,Remained stable,137,kilograms,Rarely (less than once/week),Rarely (less than once/week),Control


In [13]:
data.specialized_diet_exclude_refined_sugars.value_counts()

Yes    190
No     190
Name: specialized_diet_exclude_refined_sugars, dtype: int64

## Metadata file creation

In [18]:
metadata = data.sample_name.to_frame()

In [19]:
metadata = metadata.rename(columns={'sample_name': 'sample-id'})

In [20]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
metadata['sex'] = data['sex']
metadata['age_cat'] = data['age_cat']
metadata['refined_sugars'] = data['specialized_diet_exclude_refined_sugars']
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000023139,female,60s,Yes
1,10317.000027811,male,40s,Yes
2,10317.000031513,female,30s,Yes
3,10317.000033280,female,60s,Yes
4,10317.000038261,female,40s,Yes
...,...,...,...,...
375,10317.000065667,female,20s,No
376,10317.000065673,female,20s,No
377,10317.000065675,female,20s,No
378,10317.000065681,female,20s,No


In [21]:
metadata['sample-id'].to_csv("ids.txt",header=None,index=None)

In [22]:
metadata.refined_sugars.value_counts()

Yes    190
No     190
Name: refined_sugars, dtype: int64

In [23]:
metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)

In [24]:
from qiime2 import Metadata
sample_metadata = Metadata.load(os.getcwd()+'/sample_metadata.tsv')

Import Fasta 99% otus containing Taxonomy

In [25]:
!qiime tools import --type 'FeatureData[Sequence]' --input-path gg_13_8_otus/rep_set/99_otus.fasta \
    --output-path rep_set.qza

Imported gg_13_8_otus/rep_set/99_otus.fasta as DNASequencesDirectoryFormat to rep_set.qza


Getting otus from sample ids with biom:

In [15]:
!export ctx=Pick_closed-reference_OTUs-Greengenes-Illumina-16S-V4-5c6506

In [ ]:
!redbiom fetch samples --from ids.txt --context $ctx --output otu_table.biom

In [26]:
!biom convert -i otu_table.biom -o otu_table.tsv --to-tsv 

In [27]:
meta_table = pd.read_csv("sample_metadata.tsv",sep="\t",dtype = {"sample-id": "str"})
meta_table = meta_table.sort_values(by="sample-id").reset_index(drop=True)

In [28]:
!qiime tools import \
  --input-path otu_table.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table.qza

Imported otu_table.biom as BIOMV210Format to feature_table.qza


In [29]:
from qiime2 import Artifact
otu_table = Artifact.load("feature_table.qza")
rep_set = Artifact.load("rep_set.qza")

In [30]:
otu_index = otu_table.view(pd.DataFrame).index.sort_values()

In [31]:
ot_df = otu_table.view(pd.DataFrame) 

In [32]:
non_match = []
for i in range(meta_table.shape[0]):
    result = [k for k in otu_index if k.startswith(meta_table["sample-id"][i])]
    if len(result) == 0:
        non_match.append(meta_table["sample-id"][i] )
    else: 
        meta_table["sample-id"][i] = meta_table["sample-id"][i] + result[0][-6:]

In [33]:
meta_table_filt = meta_table.loc[~meta_table["sample-id"].isin(non_match)] 
    

In [34]:
meta_table_filt

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
373,10317.000109791.74729,male,20s,Yes
374,10317.000109807.76952,female,40s,Yes
375,10317.000110177.74729,male,30s,Yes
376,10317.000112912.76952,female,50s,Yes


In [35]:
meta_table_filt.to_csv("sample_metadata_filt.tsv",sep="\t",index=None)

In [36]:
ot_df_filt = ot_df.loc[ot_df.index.isin(meta_table_filt["sample-id"])]

In [37]:
ot_df_filt.to_csv("otu_table_filt.tsv",sep="\t")

In [38]:
!biom convert -i otu_table_filt.tsv -o otu_table_filt.biom --table-type="OTU table" --to-hdf5

In [39]:
!qiime feature-table filter-seqs --i-data rep_set.qza --m-metadata-file otu_table.tsv --o-filtered-data rep_set_filt.qza

Saved FeatureData[Sequence] to: rep_set_filt.qza


In [135]:
!biom convert -i otu_table_reorder.tsv -o otu_table_reorder.biom --table-type="OTU table" --to-hdf5

In [40]:
otu_df = otu_table.view(pd.DataFrame)

OTU filtering

In [41]:
otu_df

,383659,227697,318777,193551,297629,4359590,356052,3392847,263834,294909,...,4344207,3251419,4414478,350503,165257,512791,1718658,577679,3568710,178957
10317.000023139.25853,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000027811.25853,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,...,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000031513.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000033280.31503,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000038261.33800,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000065667.26478,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065673.25853,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065675.25853,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000065681.25853,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
otu_df = otu_df.loc[otu_df.index.isin(meta_table_filt["sample-id"])].sort_index()

In [43]:
otu_df.to_csv("otu_match.tsv", sep='\t')

In [44]:
!biom convert -i otu_match.tsv -o otu_match.biom --table-type="OTU table" --to-hdf5

In [45]:
!qiime tools import \
  --input-path otu_match.biom\
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path feature_table_match.qza

Imported otu_match.biom as BIOMV210Format to feature_table_match.qza


In [53]:
!qiime feature-table transpose --i-table feature_table_match.qza --o-transposed-feature-table feature_table_match_trans.qza

Saved FeatureTable[Frequency] to: feature_table_match_trans.qza


In [3]:
from qiime2.plugins import alignment, phylogeny
mafft_alignment = alignment.methods.mafft(rep_set)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree = phylogeny.methods.midpoint_root(unrooted_tree.tree)


NameError: name 'rep_set' is not defined

In [28]:
rooted_tree.rooted_tree.save("rooted_tree.qza")

'rooted_tree.qza'

In [47]:
from qiime2 import Artifact
rooted_tree = Artifact.load("rooted_tree.qza")

In [48]:
otu_table = Artifact.load("feature_table_match.qza")

In [49]:
from qiime2.plugins import feature_table
output_viz = feature_table.visualizers.summarize(otu_table)
output_viz.visualization.save("otu_first_analysis.qzv")

'otu_first_analysis.qzv'

In [70]:
from qiime2 import Metadata
metadata = Metadata.load("sample_metadata_filt.tsv")

In [64]:
metadata = pd.read_csv('sample_metadata_filt.tsv', sep='\t')
metadata

,sample-id,sex,age_cat,refined_sugars
0,10317.000001166.25853,female,40s,No
1,10317.000003100.69016,female,40s,No
2,10317.000004936.25853,male,50s,No
3,10317.000010862.25853,male,20s,No
4,10317.000012018.56079,female,50s,No
...,...,...,...,...
354,10317.000109791.74729,male,20s,Yes
355,10317.000109807.76952,female,40s,Yes
356,10317.000110177.74729,male,30s,Yes
357,10317.000112912.76952,female,50s,Yes


In [65]:
metadata["sample-id"]

0      10317.000001166.25853
1      10317.000003100.69016
2      10317.000004936.25853
3      10317.000010862.25853
4      10317.000012018.56079
               ...          
354    10317.000109791.74729
355    10317.000109807.76952
356    10317.000110177.74729
357    10317.000112912.76952
358    10317.000113026.81168
Name: sample-id, Length: 359, dtype: object

In [51]:
import biom

b = biom.load_table("otu_match.biom")
b

359 x 6904 <class 'biom.table.Table'> with 179471 nonzero entries (7% dense)

In [59]:
otu_table_t = Artifact.load("feature_table_match_trans.qza")

In [69]:
np.sum(otu_table_t.view(pd.DataFrame).columns == metadata["sample-id"])

359

In [71]:
from qiime2.plugins import diversity
core_metrics = diversity.pipelines.core_metrics_phylogenetic(table = otu_table_t,
                                                             phylogeny = rooted_tree,
                                                             sampling_depth = 8200,
                                                             metadata = metadata)

/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.056619661459254594 and the largest is 9.316356307490718.
  warn(
/opt/anaconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of

ValueError: None of the sample identifiers match between the metadata and the coordinates. Verify that you are using metadata and coordinates corresponding to the same dataset.

In [ ]:
import pandas as pd

In [ ]:
core_metrics.rarefied_table.view(pd.DataFrame)

,2941399,4425663,353737,363646,1090458,922239,845273,185021,293665,4453060,...,4356697,213762,179905,4324040,333726,199147,1808779,173916,435264,325586
10317.000001166.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000003100.69016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000010862.25853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000012018.56079,0.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000013727.69016,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.000109095.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109791.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000109807.76952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10317.000110177.74729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
core_metrics_rarefied_table_df = core_metrics.rarefied_table.view(pd.DataFrame).reset_index().melt(id_vars=['index'],
                                                          var_name="feature", 
                                                          value_name="Value")
core_metrics_rarefied_table_df.to_csv('core_metrics_rarefied_table_df.csv')
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [ ]:
core_metrics_rarefied_table_df

,index,feature,Value
0,10317.000001166.25853,2941399,0.0
1,10317.000003100.69016,2941399,0.0
2,10317.000010862.25853,2941399,0.0
3,10317.000012018.56079,2941399,0.0
4,10317.000013727.69016,2941399,0.0
...,...,...,...
1785987,10317.000109095.74729,325586,0.0
1785988,10317.000109791.74729,325586,0.0
1785989,10317.000109807.76952,325586,0.0
1785990,10317.000110177.74729,325586,0.0


In [ ]:
observed_otus_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.observed_otus_vector,
                                                                                     metadata)
observed_otus_vector_group_significance.visualization

AttributeError: 'Results' object has no attribute 'observed_otus_vector'

In [ ]:
shannon_vector_group_significance = diversity.actions.alpha_group_significance(core_metrics.shannon_vector,
                                                                         metadata)
shannon_vector_group_significance.visualization.save("shannon.qzv")

'shannon.qzv'

In [ ]:
faith_pd_group_significance = diversity.actions.alpha_group_significance(core_metrics.faith_pd_vector,
                                                                         metadata)
faith_pd_group_significance.visualization.save("faith_pd.qzv")

'faith_pd.qzv'

In [ ]:
evenness_group_significance = diversity.actions.alpha_group_significance(core_metrics.evenness_vector,
                                                                         metadata)
evenness_group_significance.visualization.save("evenness_group.qzv")

'evenness_group.qzv'

In [ ]:
bray_curtis_distance_significance = diversity.actions.beta_group_significance(core_metrics.bray_curtis_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
bray_curtis_distance_significance.visualization.save("bray_curtis.qzv")

'bray_curtis.qzv'

In [ ]:
jaccard_distance_significance = diversity.actions.beta_group_significance(core_metrics.jaccard_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
jaccard_distance_significance.visualization.save("jaccard_beta.qzv")

'jaccard_beta.qzv'

In [ ]:
unweighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.unweighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
unweighted_unifrac_significance.visualization.save("unweighted_unifrac.qzv")

'unweighted_unifrac.qzv'

In [ ]:
weighted_unifrac_significance = diversity.actions.beta_group_significance(core_metrics.weighted_unifrac_distance_matrix,
                                                                           metadata.get_column('refined_sugars'))
weighted_unifrac_significance.visualization.save("weighted_unifrac.qzv")

'weighted_unifrac.qzv'

In [ ]:
from qiime2.plugins import emperor
emperor_plot = emperor.visualizers.plot(core_metrics.unweighted_unifrac_pcoa_results,
                                        metadata)
                                        #custom_axes=['DaysSinceExperimentStart'])
emperor_plot.visualization.save('emperor.qzv')

'emperor.qzv'